# EV and Infrastructure 

Jubran Jalal

### Table of Contents

1. [Introduction](#introduction)
    - [Dictionary](#dictionary)
    - [Installation & Importing](#installation--import)
    - [Functions](#functions)
2. [Exploration](#exploration)
    - [Reading Datasets](#reading-datasets)
    - [Exploring Cleaning Dataset](#exploring--cleaning-the-dataset)
3. [World Insight](#world-insight)
    - [EV market world performance](#ev-market-world-performance)
    - [Outcomes of EV on the market](#outcomes-of-ev-on-the-market)
    - [World EV charging points](#world-ev-charging-points)
4. [Country Insight](#country-insight)
    - [EV sales by country](#ev-sales-by-country)
    - [EV stock by country](#ev-stock-by-country)
    - [EV charging points](#ev-charging-points-by-country)
    - [2021: Overview](#filter-to-2021-entries-and-add-bahrain)
    - [Country Comparison](#ev-charging-point-per-area-vs-population-density)


# Introduction

The critical change in climate and push towards sustainability in all types of industries. Oil-fueled vehicles are contributing to these changes. In the past years, we have noticed more attention and investment in electric vehicles and infrastructure and now it is a growing market.

__Hypothisis :__
_Bahrian should focus on investment in EV and its infrastructure_

__Question:__
- _How is EV performing in the market?_
- _What are the Outcomes of EVs in the market?_
- _Are EV infrastructures growing as the EV? (readiness)_
- _What is the EV charging point density? and how does it compare to population density?_
- _Where does Bahrain stand compared to other countries? (in terms of EV Charging Points)_

### Dictionary
__Key:__
- Electric Vehicles (EV)
- Battery Electric Vehicles (BEV)
- Plug-in Hybrid Electric Vehicle (PHEV)

__Terms:__
- EV stock: EV market stock
- EV stock share: EV stock share in %
- EV sales: sales of EV
- EV sales share: EV sales share in %
- Electricity demand: Demand for electricity by EV GWh (in Gigawatt hours)
- Oil displacement Mbd: The amount of Oil displaced by EV (in million barrels per day)
- Oil displacement Mlge: The amount of Oil displaced by EV (in million liters gasoline equivalent)

### Installation & Import

In [248]:
import pandas as pd
pd.set_option('display.max_columns',None)
import numpy as np
import requests as req
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Functions

In [249]:
# Checks the data set for null values
def check(df,show_all=False):
    '''
        check(df,show_all=False) -> None

        Checks the columns of the Dataframe for null and prints the result including the percentage of nulls.
        - show_all= default: False - displays only the columns with null values, True displays all columns with the number of null values.
    '''
    
    print(f"Number of Columns: {df.shape[1]}\nNumber of Rows: {df.shape[0]}\n")
    for col in df.columns:
        null_count=df[col].isnull().sum()
        if show_all:
            # With and without Null
            print(f"{col+':':<12}{null_count} Null values ({round(100*null_count/df.shape[0],3)}%), {'Type:':<4} ({(df[col].dtype)}), {'Unique value(s):':<4} {(df[col].value_counts().shape[0])}")
        else:
            # Only with Null
            if null_count>0:
                print(f"{col+':':<12}{null_count} Null values ({round(100*null_count/df.shape[0],3)}%), {'Type:':<4} ({(df[col].dtype)}), {'Unique value(s):':<4} {(df[col].value_counts().shape[0])}")

# Exploration

### Reading Datasets

In [250]:
# Read IEA-EV-data.csv
iea_ev_data=pd.read_csv('IEA-EV-data.csv')
iea_ev_data.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,sales,49.00000
1,Australia,Historical,EV stock,Cars,BEV,2011,stock,49.00000
2,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00046
3,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
4,Australia,Historical,EV sales share,Cars,EV,2012,percent,0.03000


#### Exploring & Cleaning the dataset: 
For Null values, data types, number of unique values, number of rows and columns, and variables

In [251]:
check(iea_ev_data, True)

Number of Columns: 8
Number of Rows: 7434

region:     0 Null values (0.0%), Type: (object), Unique value(s): 34
category:   0 Null values (0.0%), Type: (object), Unique value(s): 3
parameter:  0 Null values (0.0%), Type: (object), Unique value(s): 8
mode:       0 Null values (0.0%), Type: (object), Unique value(s): 6
powertrain: 0 Null values (0.0%), Type: (object), Unique value(s): 5
year:       0 Null values (0.0%), Type: (int64), Unique value(s): 14
unit:       0 Null values (0.0%), Type: (object), Unique value(s): 7
value:      0 Null values (0.0%), Type: (float64), Unique value(s): 961


In [252]:
# Further explore the variables with unique values lower than 10
print(iea_ev_data['category'].value_counts(),'\n')
print(iea_ev_data['parameter'].value_counts(),'\n')
print(iea_ev_data['mode'].value_counts(),'\n')
print(iea_ev_data['powertrain'].value_counts(),'\n')
print(iea_ev_data['unit'].value_counts(),'\n')
print(iea_ev_data['region'].value_counts())


Historical          5686
Projection-STEPS     874
Projection-APS       874
Name: category, dtype: int64 

EV stock                 1723
EV sales                 1609
EV stock share           1096
EV sales share           1059
EV charging points        623
Electricity demand        576
Oil displacement Mbd      374
Oil displacement Mlge     374
Name: parameter, dtype: int64 

Cars         2684
Vans         1625
Buses        1436
Trucks       1058
EV            623
SSPACEHHP       8
Name: mode, dtype: int64 

EV                         2903
BEV                        2373
PHEV                       1535
Publicly available slow     313
Publicly available fast     310
Name: powertrain, dtype: int64 

percent                              2155
stock                                1723
sales                                1609
charging points                       623
GWh                                   576
Milion barrels per day                374
Milion litres gasoline equivalent     374


In [253]:
# Inspecting dataset by filtering 'category'
iea_ev_data[(iea_ev_data['category']=='Historical')&(iea_ev_data['parameter']=='EV sales')].head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,sales,49.0
7,Australia,Historical,EV sales,Cars,PHEV,2012,sales,80.0
8,Australia,Historical,EV sales,Cars,BEV,2012,sales,170.0
11,Australia,Historical,EV sales,Cars,BEV,2013,sales,190.0
12,Australia,Historical,EV sales,Cars,PHEV,2013,sales,100.0


Upon analyzing the dataset:
- The dataset can be divided into two separate datasets 'Historical' data and 'Projection' data. The focus of this Analysis will be on the 'Historical' data.
- The 'Historical' data contains data on Electric vehicle sales (EV sales), Electric vehicle sales share (EV sales share), Electric vehicle stock (EV stock), Electric vehicle stock share (EV stock share), Electric vehicle charging points (EV charging points), Oil displacement Mbd, Oil displacement Mlge
- Sales and Stocks further branch out into the type of Vehicles such as Cars, Buses, Vans, Trucks
- The 'unit' refers to the unit of the 'value'
- 'region' refers to countries. However, it also includes 'World'.

In [254]:
# Seperate the projection from historical
iea_ev_data_proj=iea_ev_data[iea_ev_data['category']!='Historical']
iea_ev_data.drop(axis=0,labels=iea_ev_data[iea_ev_data['category']!='Historical'].index,inplace=True)

In [255]:
# Since the category has 1 value and is of no use to the analysis we drop it
iea_ev_data.drop(axis=1,columns=['category'],inplace=True)
iea_ev_data.shape

(5686, 7)

## World Insight:

In [256]:
# Exracting & reordering 'World' records from 'iea_ev_data'
world_ev_data = iea_ev_data[(iea_ev_data['region']=='World') & (iea_ev_data['parameter']=='EV sales')].groupby(['year'])['value'].sum().reset_index().rename(columns={'value':'Total EV sales'}).set_index('year').join(\
iea_ev_data[(iea_ev_data['region']=='World') & (iea_ev_data['parameter']=='EV stock')].groupby(['year'])['value'].sum().reset_index().rename(columns={'value':'Total EV stock'}).set_index('year'),on='year').join(\
iea_ev_data[(iea_ev_data['region']=='World') & (iea_ev_data['parameter']=='EV charging points')].groupby(['year'])['value'].sum().reset_index().rename(columns={'value':'Total EV charging points'}).set_index('year'),on='year').join(\
iea_ev_data[(iea_ev_data['region']=='World') & (iea_ev_data['parameter']=='Oil displacement Mbd')].groupby(['year'])['value'].sum().reset_index().rename(columns={'value':'Total Oil displacement Mbd'}).set_index('year'),on='year').join(\
iea_ev_data[(iea_ev_data['region']=='World') & (iea_ev_data['parameter']=='Electricity demand')].groupby(['year'])['value'].sum().reset_index().rename(columns={'value':'Total Electricity demand'}).set_index('year'),on='year').join(\
iea_ev_data[(iea_ev_data['region']=='World') & (iea_ev_data['parameter']=='Oil displacement Mlge')].groupby(['year'])['value'].sum().reset_index().rename(columns={'value':'Total Oil displacement Mlge'}).set_index('year'),on='year')

In [257]:
world_ev_data.head(1)

,Total EV sales,Total EV stock,Total EV charging points,Total Oil displacement Mbd,Total Electricity demand,Total Oil displacement Mlge
year,,,,,,
2010,11401.0,24190.0,4010.0,0.001295,NaN,75.579


#### __EV market world performance__

#### World EV sales

In [258]:
# Graph the EV sales from 2010 - 2021
ev_sales_fig=px.line(world_ev_data,x=world_ev_data.index,y='Total EV sales',markers=True, title='The growth of EV sales')
ev_sales_fig.show()

In [259]:
# Extract the 'World' sales data by type of Vehicle
world_sales_details=iea_ev_data[((iea_ev_data['year']>=2019)&(iea_ev_data['year']<=2021))&(iea_ev_data['region']=='World')&(iea_ev_data['parameter']=='EV sales')]
world_sales_details.drop(columns=['region','parameter','unit'],inplace=True)
world_sales_details=world_sales_details[world_sales_details['powertrain']=='BEV'].merge(world_sales_details[world_sales_details['powertrain']=='PHEV'],how='outer',on=['year','mode']).rename(columns={'value_x':'BEV','value_y':'PHEV'})
world_sales_details.drop(columns=['powertrain_x','powertrain_y'],inplace=True)
world_sales_details.head()

C:\Users\jubra\AppData\Local\Temp\ipykernel_3052\2546250560.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,mode,year,BEV,PHEV
0,Buses,2019,100000.0,2600.0
1,Trucks,2019,17000.0,2400.0
2,Vans,2019,53000.0,270.0
3,Cars,2019,1500000.0,580000.0
4,Buses,2020,61000.0,3700.0


In [260]:
check(world_sales_details,True)

Number of Columns: 4
Number of Rows: 12

mode:       0 Null values (0.0%), Type: (object), Unique value(s): 4
year:       0 Null values (0.0%), Type: (int64), Unique value(s): 3
BEV:        0 Null values (0.0%), Type: (float64), Unique value(s): 12
PHEV:       0 Null values (0.0%), Type: (float64), Unique value(s): 12


Exploring the sales, by type of EV from 2019 - 2021 

In [261]:
#Graph of global sales of BEV by type  from 2019 - 2021
fig=px.bar(world_sales_details,x='year',y='BEV',color='mode',barmode='group',title='Global sales of BEV by type  from 2019 - 2021')
fig.show()

In [262]:
#Graph of global sales of PHEV by type  from 2019 - 2021
fig=px.bar(world_sales_details,x='year',y='PHEV',color='mode',barmode='group',title='Global sales of PHEV by type  from 2019 - 2021')
fig.show()

The sales of EVs have increased substantially between 2018 - 2021, especially between 2020 - 2021. Over the past three years, car sales are leading compared to vans, buses, and trucks. The BEV holds have more preference over the PHEV. 

#### World EV stock

In [263]:
# Graph the EV stock from 2010 - 2021
ev_sales_fig=px.line(world_ev_data,x=world_ev_data.index,y='Total EV stock',markers=True, title='The growth of EV stock')
ev_sales_fig.show()

EV stock shows continuous growth in the EV market.

#### __Outcomes of EV on the market__

#### World Oil displacement (Mbd)

In [264]:
# Graph the Oil displacement (Mbd) from 2010 - 2021
ev_sales_fig=px.line(world_ev_data,x=world_ev_data.index,y='Total Oil displacement Mbd',markers=True, title='Change in Oil displacement (Mbd)')
ev_sales_fig.show()

The number of displaced oil barrels per day by EVs increased as the demand for EVs increased.

#### World electricty demand by EV

In [265]:
# Graph the Electricty demand GWh from 2015 - 2021
ev_sales_fig=px.line(world_ev_data,x=world_ev_data.index,y='Total Electricity demand',markers=True, title='Change Electricity demand')
ev_sales_fig.show()

A significant increase in the electricity demand with the increase in EV sales.

#### World EV Charging points

In [266]:
# Graph the EV charging points from 2010 - 2021
ev_sales_fig=px.line(world_ev_data,x=world_ev_data.index,y='Total EV charging points',markers=True, title='The growth of EV charging points')
ev_sales_fig.show()

With the increase in EV sales, the number of EV charging points increased to support the country's EV infrastructure.

## Country Insight

In [267]:
# Filtering out 'World' records from 'iea_ev_data'
iea_ev_data=iea_ev_data[iea_ev_data['region']!='World']

In [268]:
iea_ev_data.head()

,region,parameter,mode,powertrain,year,unit,value
0,Australia,EV sales,Cars,BEV,2011,sales,49.00000
1,Australia,EV stock,Cars,BEV,2011,stock,49.00000
2,Australia,EV stock share,Cars,EV,2011,percent,0.00046
3,Australia,EV sales share,Cars,EV,2011,percent,0.00650
4,Australia,EV sales share,Cars,EV,2012,percent,0.03000


Adding Population density, each country's land area, and geolocation.

In [269]:
# Country Land area for calculating: Total EV 
country_land=pd.read_csv('land-area-km.csv')
# Filter and Clean 'country_land' : Year -> 2021, Drop -> 'Code'
country_land=country_land[country_land['Year']==2021].reset_index(drop=True)
country_land.drop(columns=['Code','Year'],inplace=True)
country_land.head()

,Entity,Land area (sq. km)
0,Afghanistan,652860.0
1,Albania,27400.0
2,Algeria,2381741.0
3,American Samoa,200.0
4,Andorra,470.0


In [270]:
country_pop_dens=pd.read_csv('WPP2022_Demographic_Indicators_Medium.csv')
# Filter and Clean 'country_pop_dens' : Year -> 2021, Location -> All except for 'World'
country_pop_dens=country_pop_dens[(country_pop_dens['Location']!='World')&(country_pop_dens['Time']==2021)][['Location','PopDensity']].reset_index(drop=True)
country_pop_dens.head()

C:\Users\jubra\AppData\Local\Temp\ipykernel_3052\2062369730.py:1: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



,Location,PopDensity
0,Sub-Saharan Africa,52.0442
1,Northern Africa and Western Asia,43.7128
2,Central and Southern Asia,199.9217
3,Eastern and South-Eastern Asia,144.0539
4,Latin America and the Caribbean,32.3747


In [271]:
country_geolocation=pd.read_csv('country_geolocation.csv',encoding='latin')
country_geolocation.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


#### EV sales by country

In [272]:
country_ev_data=iea_ev_data[iea_ev_data['parameter']=='EV sales'][['year','region','value']].groupby(\
                        ['year','region'])['value'].sum().reset_index().rename(columns={'value':'Total EV sales'})
country_ev_data.head()

,year,region,Total EV sales
0,2010,Belgium,42.0
1,2010,China,3464.9
2,2010,Denmark,62.0
3,2010,France,1006.0
4,2010,Germany,310.0


#### EV stock by country

In [273]:
country_ev_data=country_ev_data.merge(iea_ev_data[iea_ev_data['parameter']=='EV stock'][['year','region','value']].groupby(\
                        ['year','region'])['value'].sum().reset_index().rename(columns={'value':'Total EV stock'}),how='outer',on=['year','region'])
country_ev_data.head()

,year,region,Total EV sales,Total EV stock
0,2010,Belgium,42.0,137.0
1,2010,China,3464.9,4240.0
2,2010,Denmark,62.0,103.0
3,2010,France,1006.0,2116.0
4,2010,Germany,310.0,470.0


#### EV charging points by country

In [274]:
country_ev_data=country_ev_data.merge(iea_ev_data[iea_ev_data['parameter']=='EV charging points'][['year','region','value']].groupby(\
                        ['year','region'])['value'].sum().reset_index().rename(columns={'value':'Total publicly available charging points'}),how='outer',on=['year','region'])
country_ev_data.head()

,year,region,Total EV sales,Total EV stock,Total publicly available charging points
0,2010,Belgium,42.0,137.0,NaN
1,2010,China,3464.9,4240.0,NaN
2,2010,Denmark,62.0,103.0,NaN
3,2010,France,1006.0,2116.0,NaN
4,2010,Germany,310.0,470.0,NaN


__Observation :__ Top 5 Countries in terms of Total of EV sales, stock, and charging points in 2021

In [275]:
# Top 5 Countries  by EV sales, EV stock, and Total publicly available charging points in 2021
country_ev_data[country_ev_data['year']==2021].sort_values(['Total EV sales','Total EV stock','Total publicly available charging points'],ascending=[False,False,False]).head(5)

,year,region,Total EV sales,Total EV stock,Total publicly available charging points
310,2021,China,3484860.0,8877730.0,1150000.0
314,2021,Germany,703997.0,1321260.0,51200.0
332,2021,USA,630000.0,2020000.0,114000.0
333,2021,United Kingdom,324877.0,781100.0,36700.0
313,2021,France,323480.0,793070.0,54500.0


In 2021, China has been leading the EV sales, EV stock, and the publicly available charging points

#### Filter to 2021 entries and add Bahrain

Bahrain has 3 EV Charging points (according to Google Maps). However, the EV sales and stock were not available.

In [276]:
# Fillter 'country_ev_2021_data' year to 2021
country_ev_data=country_ev_data[country_ev_data['year']==2021].reset_index(drop=True)
country_ev_data.head()

,year,region,Total EV sales,Total EV stock,Total publicly available charging points
0,2021,Australia,20400.0,48000.0,2350.0
1,2021,Belgium,70806.0,174791.0,12790.0
2,2021,Brazil,7200.0,12700.0,454.9
3,2021,Canada,92450.0,308637.0,15100.0
4,2021,Chile,570.0,1400.0,480.0


In [277]:
# Add Bahrain to the list. The number of Charging Points is 3 (According to Google Maps)
country_ev_data.loc[len(country_ev_data.index)]=[2021,'Bahrain',np.nan,np.nan,3.0]

In [278]:
country_ev_data.tail()

,year,region,Total EV sales,Total EV stock,Total publicly available charging points
27,2021,USA,630000.0,2020000.0,114000.0
28,2021,United Kingdom,324877.0,781100.0,36700.0
29,2021,Indonesia,NaN,NaN,184.0
30,2021,Thailand,NaN,NaN,2270.0
31,2021,Bahrain,NaN,NaN,3.0


### EV Charging Point per Area vs. Population Density

In [279]:
country_ev_2021_data=country_ev_data[country_ev_data['year']==2021]

Joining Land area from country_land to country_ev_data

In [280]:
# Check the countries in the 'ev_chargingpoint_country' if exist in 'country_land'
for country in country_ev_2021_data['region']:
    if not country in country_land['Entity'].tolist():
        print(country)

Korea
Other Europe
USA


In [281]:
# Ignore: Other Europe
# Replace: Korea -> South Korea in 'ev_chargingpoint_country'
country_ev_2021_data['region'].replace({'Korea':'South Korea'},inplace=True)
# Replace: United States -> USA in 'country_land'
country_land['Entity'].replace({'United States':'USA'},inplace=True)

In [282]:
# Check the countries in the 'ev_chargingpoint_country' if exist in 'country_pop_dens'
for country in country_ev_2021_data['region']:
    if not country in country_pop_dens['Location'].tolist():
        print(country)

South Korea
Other Europe
USA


In [283]:
# Ignore: Other Europe
# Replace: Republic of Korea -> South Korea in 'country_pop_dens'
country_pop_dens['Location'].replace({'Republic of Korea':'South Korea'},inplace=True)
# Replace: United States of America -> USA in 'country_pop_dens'
country_pop_dens['Location'].replace({'United States of America':'USA'},inplace=True)

In [284]:
# Check the countries in the 'ev_chargingpoint_country' if exist in 'country_pop_dens'
for country in country_ev_2021_data['region']:
    if not country in country_geolocation['name'].tolist():
        print(country)

Other Europe
USA


In [285]:
# Ignore: Other Europe
# Replace: United States of America -> USA in 'country_pop_dens'
country_geolocation['name'].replace({'United States':'USA'},inplace=True)

In [286]:
# Join 'ev_chargingpoint_country' with 'country_land'
ev_chargept_landarea_popdens=country_ev_2021_data.merge(country_land,how='left',left_on='region',right_on='Entity')
ev_chargept_landarea_popdens.drop(columns='Entity',inplace=True)

ev_chargept_landarea_popdens.head()

,year,region,Total EV sales,Total EV stock,Total publicly available charging points,Land area (sq. km)
0,2021,Australia,20400.0,48000.0,2350.0,7692020.0
1,2021,Belgium,70806.0,174791.0,12790.0,30280.0
2,2021,Brazil,7200.0,12700.0,454.9,8358140.0
3,2021,Canada,92450.0,308637.0,15100.0,8965590.0
4,2021,Chile,570.0,1400.0,480.0,743532.0


In [287]:
# Add Column with a calculated value of Total Charging Points / Land Area
ev_chargept_landarea_popdens['Charging points per Land Area']=ev_chargept_landarea_popdens['Total publicly available charging points']/ev_chargept_landarea_popdens['Land area (sq. km)']
ev_chargept_landarea_popdens.head()

,year,region,Total EV sales,Total EV stock,Total publicly available charging points,Land area (sq. km),Charging points per Land Area
0,2021,Australia,20400.0,48000.0,2350.0,7692020.0,0.000306
1,2021,Belgium,70806.0,174791.0,12790.0,30280.0,0.422391
2,2021,Brazil,7200.0,12700.0,454.9,8358140.0,0.000054
3,2021,Canada,92450.0,308637.0,15100.0,8965590.0,0.001684
4,2021,Chile,570.0,1400.0,480.0,743532.0,0.000646


In [288]:
# Join 'ev_chargingpoint_country' with 'country_land'
ev_chargept_landarea_popdens=ev_chargept_landarea_popdens.merge(country_pop_dens,how='left',left_on='region',right_on='Location')
ev_chargept_landarea_popdens.drop(columns='Location',inplace=True)
ev_chargept_landarea_popdens.rename(columns={'PopDensity':'Population density (people per sq km)'},inplace=True)
ev_chargept_landarea_popdens.head()

,year,region,Total EV sales,Total EV stock,Total publicly available charging points,Land area (sq. km),Charging points per Land Area,Population density (people per sq km)
0,2021,Australia,20400.0,48000.0,2350.0,7692020.0,0.000306,3.3741
1,2021,Belgium,70806.0,174791.0,12790.0,30280.0,0.422391,383.4936
2,2021,Brazil,7200.0,12700.0,454.9,8358140.0,0.000054,25.6428
3,2021,Canada,92450.0,308637.0,15100.0,8965590.0,0.001684,4.1959
4,2021,Chile,570.0,1400.0,480.0,743532.0,0.000646,25.8889


In [289]:
# Join 'country_geolocation' with 'ev_chargingpoint_country'
ev_chargept_landarea_popdens=ev_chargept_landarea_popdens.merge(country_geolocation,how='left',left_on='region',right_on='name')
ev_chargept_landarea_popdens.drop(columns='name',inplace=True)
ev_chargept_landarea_popdens.rename(columns={'country':'ISOCODE2'},inplace=True)
ev_chargept_landarea_popdens.head()

,year,region,Total EV sales,Total EV stock,Total publicly available charging points,Land area (sq. km),Charging points per Land Area,Population density (people per sq km),ISOCODE2,latitude,longitude
0,2021,Australia,20400.0,48000.0,2350.0,7692020.0,0.000306,3.3741,AU,-25.274398,133.775136
1,2021,Belgium,70806.0,174791.0,12790.0,30280.0,0.422391,383.4936,BE,50.503887,4.469936
2,2021,Brazil,7200.0,12700.0,454.9,8358140.0,0.000054,25.6428,BR,-14.235004,-51.925280
3,2021,Canada,92450.0,308637.0,15100.0,8965590.0,0.001684,4.1959,CA,56.130366,-106.346771
4,2021,Chile,570.0,1400.0,480.0,743532.0,0.000646,25.8889,CL,-35.675147,-71.542969


In [293]:
ev_chargept_landarea_popdens[ev_chargept_landarea_popdens['region']=='South Korea']

,year,region,Total EV sales,Total EV stock,Total publicly available charging points,Land area (sq. km),Charging points per Land Area,Population density (people per sq km),ISOCODE2,latitude,longitude
15,2021,South Korea,119700.0,287600.0,107000.0,97520.0,1.097211,523.8384,KR,35.907757,127.766922


In [300]:
fig = px.bar(ev_chargept_landarea_popdens[ev_chargept_landarea_popdens['region']!='Other Europe'].sort_values(by='Population density (people per sq km)',ascending=True), x="Population density (people per sq km)", y="region", orientation='h',
             title='Population density (people per sq km)',height=700)
fig.show()

In [299]:
fig = px.bar(ev_chargept_landarea_popdens[ev_chargept_landarea_popdens['region']!='Other Europe'].sort_values(by='Charging points per Land Area',ascending=True), 
             x="Charging points per Land Area", y="region", orientation='h',title='EV charging points per Land area',height=700)
fig.show()

## __" _Bahrain should invest in EV and its infrastructure_ "__